In [ ]:
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from xgboost import XGBClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
import pandas as pd
from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np
import time
from imblearn.under_sampling import RandomUnderSampler


In [ ]:
class Dataset():
    def __init__(self):

        # whole dataset
        df = pd.read_csv('dataset_2.csv')
        self.y = df['K2Q31A']
        self.X = df.drop(columns='K2Q31A')

        # chi square
        self.chi = pd.read_csv('chi.csv')
        self.chi = self.chi.loc[self.chi['Dr Sheikhy'] == 'Y']['feature name']

        # fisher's score
        self.fisher = pd.read_csv('fisher.csv')
        self.fisher = self.fisher.loc[self.fisher['Dr Sheikhy'] == 'Y']['Feature Name']

        # information gain
        self.inf = pd.read_csv('inf-gain.csv')
        self.inf = self.inf.loc[self.inf['Dr Sheikhy'] == 'Y']['Feature Name']

        # corelation
        self.cor = pd.read_csv('/cor.csv')
        self.cor = self.cor.iloc[:,[0, 14]].where(self.cor.iloc[:,14] == 'Y').dropna().iloc[:, 0]

    def return_dataset(self) -> pd.DataFrame:
        return self.X, self.y

    def return_chi(self) -> pd.Series:
        return self.chi

    def return_fisher(self) -> pd.Series:
        return self.fisher

    def return_inf(self) -> pd.Series:
        return self.inf

    def return_cor(self) -> pd.Series:
        return self.cor

    # intersections of 2 sets
    def return_intersection_chi_fisher(self) -> list:
        return list(set(self.chi.tolist()) & set(self.fisher.tolist()))

    def return_intersection_chi_inf(self) -> list:
        return list(set(self.chi.tolist()) & set(self.inf.tolist()))

    def return_intersection_chi_cor(self) -> list:
        return list(set(self.chi.tolist()) & set(self.cor.tolist()))

    def return_intersection_fisher_inf(self) -> list:
        return list(set(self.fisher.tolist()) & set(self.inf.tolist()))

    def return_intersection_fisher_cor(self) -> list:
        return list(set(self.fisher.tolist()) & set(self.cor.tolist()))

    def return_intersection_inf_cor(self) -> list:
        return list(set(self.inf.tolist()) & set(self.cor.tolist()))

    # intersections of 3 sets
    def return_intersection_chi_fisher_inf(self) -> list:
        return list(set(self.chi.tolist()) & set(self.fisher.tolist()) & set(self.inf.tolist()))

    def return_intersection_chi_fisher_cor(self) -> list:
        return list(set(self.chi.tolist()) & set(self.fisher.tolist()) & set(self.cor.tolist()))

    def return_intersection_chi_inf_cor(self) -> list:
        return list(set(self.chi.tolist()) & set(self.inf.tolist()) & set(self.cor.tolist()))

    def return_intersection_fisher_inf_cor(self) -> list:
        return list(set(self.fisher.tolist()) & set(self.inf.tolist()) & set(self.cor.tolist()))




In [ ]:
class Model():
    def __init__(self, no_iters: int, selection: list, x: pd.DataFrame, y: pd.DataFrame):

        self.no_iters = no_iters

        self.x = x.loc[:, x.columns.isin(selection)]
        self.y = y

        # [name, acc, f1_score, recall, precision, time]
        self.xgboost_result = []
        self.adaboost_result = []
        self.gradient_boost_result = []
        self.random_forest_result = []
        self.svm_result = []

        # classifiers
        self.xgboost_classifier = XGBClassifier(max_depth=5,
        learning_rate=0.005,
        n_estimators=100,
        objective='binary:logistic',
        random_state=42)

        self.adaboost_classifier = AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2),
        n_estimators=100,
        learning_rate=0.05,
        algorithm='SAMME.R',
        random_state=42,
        base_estimator='deprecated')

        self.svm_classifier = SVC()

        self.random_forest_classifier = RandomForestClassifier()

        self.gradient_boost_classifier = GradientBoostingClassifier()

        # undersampling
        self.undersample = RandomUnderSampler(sampling_strategy='majority')

    def _append_items(self, target: list, name, acc, f1_score, recall, precision, time, no_iters):
        target.append(name)
        target.append(acc)
        target.append(f1_score)
        target.append(recall)
        target.append(precision)
        target.append(time)
        target.append(no_iters)
        return target

    def _train(self, classifier, random_state=0):

        X, y = self.undersample.fit_resample(self.x, self.y)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=random_state)

        y_train = -(y_train - 2)
        y_test = -(y_test - 2)

        start_time = time.time()

        classifier.fit(X_train, y_train)

        y_pred = classifier.predict(X_test)

        end_time = time.time()

        time0 = end_time - start_time
        accuracy = accuracy_score(y_test, y_pred) * 100
        f1 = f1_score(y_test, y_pred)
        recall = recall_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred)



        return accuracy, f1, recall, precision, time0
    # [name, acc, f1_score, recall, precision, time]


    def display_result(self, result, filename='result.text', print_result=True):
        with open(filename, 'a') as file:
            result_string = (f'algorithm name: {result[0]}\n'
                f'number of iterations: {str(self.no_iters)}\n'
                f'average accuracy: {(result[1]).mean():.4f}' + '\n'
                f'max accuracy: {(result[1]).max():.4f}' + '\n'
                f'std accuracy: {(result[1]).std():.4f}' + '\n'
                f'average f1 score: {(result[2]).mean():.4f}' + '\n'
                f'max f1 score: {(result[2]).max():.4f}' + '\n'
                f'std f1 score: {(result[2]).std():.4f}' + '\n'
                f'average recall: {(result[3]).mean():.4f}' + '\n'
                f'max recall: {(result[3]).max():.4f}' + '\n'
                f'std recall: {(result[3]).std():.4f}' + '\n'
                f'average precision: {(result[4]).mean():.4f}' + '\n'
                f'max precision: {(result[4]).max():.4f}' + '\n'
                f'std precision: {(result[4]).std():.4f}' + '\n'
                f'average time: {result[5].mean():.4f}' + '\n\n\n')
            if print_result:
                print(result_string)
            file.write(result_string)
        file.close()


    def fit_algorithm(self, classifier):

        scores = []
        times = []
        f1_scores = []
        recall_scores = []
        precision_scores = []

        for _ in range(self.no_iters):

            accuracy, f1, recall, precision, time0 = self._train(classifier)

            scores.append(accuracy)
            f1_scores.append(f1)
            recall_scores.append(recall)
            precision_scores.append(precision)
            times.append(time0)

        target = []


        name = classifier
        if  name == self.xgboost_classifier:
            name = XGBClassifier.__name__


        target = self._append_items(target=target, name=name, acc=np.array(scores),
                            f1_score=np.array(f1_scores), recall=np.array(recall_scores),
                            precision=np.array(precision_scores), time=np.array(times), no_iters=self.no_iters)
        self.display_result(target)



In [ ]:
no_iterarion =200

In [ ]:
dataset  = Dataset()
X, y = dataset.return_dataset()

model = Model(no_iters=no_iterarion, selection=dataset.return_intersection_chi_inf(), x=X, y=y)


In [ ]:
classifiers =   [
                model.xgboost_classifier,
                model.adaboost_classifier,
                model.gradient_boost_classifier,
                model.random_forest_classifier,
                model.svm_classifier
                ]

In [ ]:
model.fit_algorithm(classifier=classifiers[0])



algorithm name: XGBClassifier
number of iterations: 200
average accuracy: 86.2832
max accuracy: 88.4007
std accuracy: 0.5957
average f1 score: 0.8601
max f1 score: 0.8797
std f1 score: 0.0056
average recall: 0.8507
max recall: 0.8617
std recall: 0.0076
average precision: 0.8699
max precision: 0.9050
std precision: 0.0108
average time: 0.9889





In [ ]:
model.fit_algorithm(classifier=classifiers[1])


algorithm name: AdaBoostClassifier(estimator=DecisionTreeClassifier(max_depth=2),
                   learning_rate=0.05, n_estimators=100, random_state=42)
number of iterations: 200
average accuracy: 86.0562
max accuracy: 87.4048
std accuracy: 0.4987
average f1 score: 0.8553
max f1 score: 0.8682
std f1 score: 0.0046
average recall: 0.8311
max recall: 0.8440
std recall: 0.0040
average precision: 0.8809
max precision: 0.9026
std precision: 0.0092
average time: 1.0049





In [ ]:
model.fit_algorithm(classifier=classifiers[2])


algorithm name: GradientBoostingClassifier()
number of iterations: 200
average accuracy: 86.4537
max accuracy: 87.9320
std accuracy: 0.6021
average f1 score: 0.8610
max f1 score: 0.8748
std f1 score: 0.0055
average recall: 0.8464
max recall: 0.8617
std recall: 0.0048
average precision: 0.8762
max precision: 0.9011
std precision: 0.0108
average time: 0.9342





In [ ]:
model.fit_algorithm(classifier=classifiers[3])


algorithm name: RandomForestClassifier()
number of iterations: 200
average accuracy: 85.2390
max accuracy: 87.4048
std accuracy: 0.5836
average f1 score: 0.8507
max f1 score: 0.8706
std f1 score: 0.0053
average recall: 0.8485
max recall: 0.8617
std recall: 0.0048
average precision: 0.8530
max precision: 0.8871
std precision: 0.0097
average time: 0.8559





In [ ]:
model.fit_algorithm(classifier=classifiers[4])


algorithm name: SVC()
number of iterations: 200
average accuracy: 84.9153
max accuracy: 86.2917
std accuracy: 0.5878
average f1 score: 0.8456
max f1 score: 0.8577
std f1 score: 0.0054
average recall: 0.8335
max recall: 0.8475
std recall: 0.0051
average precision: 0.8582
max precision: 0.8852
std precision: 0.0099
average time: 1.9673



